In [7]:
from dotenv import load_dotenv
import os
from pypdf import PdfReader

In [8]:
load_dotenv()
GOOGLW_API_KEY = os.getenv("GOOGLE_API_KEY")

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory

In [11]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [13]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 100)
    chunks = text_splitter.split_text(text)
    return chunks

In [19]:
GoogleGenerativeAIEmbeddings(model="textembedding-gecko@001", google_api_key=GOOGLW_API_KEY)

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f19dc99f050>, model='textembedding-gecko@001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [25]:
def get_vector_store(chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="textembedding-gecko@001", google_api_key=GOOGLW_API_KEY)
    vector_store = Chroma.from_texts(chunks, embedding=embeddings, persist_directory="./db")
    return get_vector_store

In [22]:
GoogleGenerativeAI(model="gemini-1.5-flash-001", google_api_key=GOOGLW_API_KEY, temperature=0.2)

GoogleGenerativeAI(model='gemini-1.5-flash-001', google_api_key=SecretStr('**********'), temperature=0.2, client=genai.GenerativeModel(
    model_name='models/gemini-1.5-flash-001',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [23]:
def get_conversational_chain(vector_store):
    llm = GoogleGenerativeAI(model="gemini-1.5-flash-001", google_api_key=GOOGLW_API_KEY, temperature=0.2)
    memory = ConversationBufferMemory(memory_key= "chat_history", return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vector_store.as_retriver(), memory=memory)
    return conversation_chain